In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import KFold

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

from sklearn.preprocessing import RobustScaler
rc = RobustScaler()

In [ ]:
train_path = "../input/ventilator-pressure-prediction/train.csv"
test_path = "../input/ventilator-pressure-prediction/test.csv"
sample_sub = "../input/ventilator-pressure-prediction/sample_submission.csv"

In [ ]:
def dropCols(df, cols):
    df = df.copy()
    df.drop(cols, axis=1, inplace=True)
    return df

In [ ]:
def preProcess(df):   
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    #df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    #df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    #df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    #df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    #df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    #df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    #df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    #df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    #df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    #df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    #df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    #df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    #df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    #df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    #df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)
    
    #df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    #df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    #df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    #df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    #df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    #df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    #df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']

    #df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    #df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    #df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    #df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    #df['cross']= df['u_in']*df['u_out']
    #df['cross2']= df['time_step']*df['u_out']
    
    #df['R'] = df['R'].astype(str)
    #df['C'] = df['C'].astype(str)
    #df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    #df = pd.get_dummies(df)
    return df

In [ ]:
train_data = pd.read_csv(train_path)
train_data = preProcess(train_data)

In [ ]:
cols_2_drop = ['id', 'breath_id', 'time_step']

In [ ]:
train_df = dropCols(train_data, cols_2_drop)

Y = train_df.pop('pressure')

In [ ]:
train_df.shape

In [ ]:
train_df.isna().sum()

In [ ]:
rc.fit(train_df)
train_df = rc.transform(train_df)

In [ ]:
train_df = train_df.reshape(-1, 80, train_df.shape[-1])
Y = Y.values.reshape(-1, 80, 1)

In [ ]:
train_df.shape, Y.shape

In [ ]:
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="C_out_loss",
    factor=0.96,
    patience=4,
    verbose=1,
)

In [ ]:
def build_model():
    
    Input = layers.Input(shape=([80, train_df.shape[-1]]))
    
    m1x1 = layers.Bidirectional(layers.LSTM(units=440, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    m1x2 = layers.Bidirectional(layers.LSTM(units=360, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    m1x3 = layers.Bidirectional(layers.LSTM(units=240, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    m1x4 = layers.Bidirectional(layers.LSTM(units=180, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    m1x5 = layers.Bidirectional(layers.LSTM(units=100, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    
    m2x1 = layers.Bidirectional(layers.LSTM(units=880, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    m2x3 = layers.Bidirectional(layers.LSTM(units=440, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))#
    m2x5 = layers.Bidirectional(layers.LSTM(units=360, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))#
    m2x7 = layers.Bidirectional(layers.LSTM(units=240, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))#
    m2x9 = layers.Bidirectional(layers.LSTM(units=180, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))#
    m2x11 = layers.Bidirectional(layers.LSTM(units=100, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    
    m3x1 = layers.Bidirectional(layers.GRU(units=440, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    m3x2 = layers.Bidirectional(layers.GRU(units=360, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    m3x3 = layers.Bidirectional(layers.GRU(units=240, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    m3x4 = layers.Bidirectional(layers.GRU(units=180, dropout=0.2, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    m3x5 = layers.Bidirectional(layers.GRU(units=100, return_sequences=True, kernel_initializer=tf.keras.initializers.GlorotNormal()))
    
    m3x1_out = m3x1(Input)
    m3x2_out = m3x2(m3x1_out)
    m3x3_out = m3x3(m3x2_out)
    m3x4_out = m3x4(m3x3_out)
    m3x5_out = m3x5(m3x4_out)
    
    
    m2x1_out = m2x1(Input)
    m2x3_out = m2x3(m2x1_out)
    m2x5_out = m2x5(m2x3_out)
    m2x7_out = m2x7(m2x5_out)
    m2x9_out = m2x9(m2x7_out)
    m2x11_out = m2x11(m2x9_out)
    
    m1x1_out = m1x1(Input)
    m1x1_out = layers.Multiply()([m1x1_out, m2x3_out, m3x1_out])
    m1x1_out = layers.BatchNormalization()(m1x1_out)
    
    m1x2_out = m1x2(m1x1_out)
    m1x2_out = layers.Multiply()([m1x2_out, m2x5_out, m3x2_out])
    m1x2_out = layers.BatchNormalization()(m1x2_out)
    
    m1x3_out = m1x3(m1x2_out)
    m1x3_out = layers.Multiply()([m1x3_out, m2x7_out, m3x3_out])
    m1x3_out = layers.BatchNormalization()(m1x3_out)
    
    m1x4_out = m1x4(m1x3_out)
    m1x4_out = layers.Multiply()([m1x4_out, m2x9_out, m3x4_out])
    m1x4_out = layers.BatchNormalization()(m1x4_out)
    
    m1x5_out = m1x5(m1x4_out)

    f_mul = layers.Multiply()([m1x5_out, m2x11_out, m3x5_out])
    f_mul = layers.BatchNormalization()(f_mul)
    f_mul = layers.Bidirectional(layers.LSTM(units=100, dropout=0.2, return_sequences=True))(f_mul)
    
    
    C_out = layers.Dense(units=64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(f_mul)
    C_out = layers.Dense(1, name="C_out", kernel_initializer=tf.keras.initializers.HeNormal())(C_out)
    
    m1_out = layers.Dense(units=64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(m1x5_out)
    m1_out = layers.Dense(1, name='m1_out', kernel_initializer=tf.keras.initializers.HeNormal())(m1_out)
        
    model = tf.keras.Model(inputs=Input, outputs=[C_out, m1_out], name='Base_Model')
    
    losses = {
        "C_out":tf.keras.losses.MeanAbsoluteError(),
        "m1_out":tf.keras.losses.MeanAbsoluteError()
    }
    
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=losses)
    
    return model

In [ ]:
test_model = build_model()
test_model.summary()

In [ ]:
tf.keras.utils.plot_model(test_model)

In [ ]:
EPOCH = 300
BATCH_SIZE = 512

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    kf = KFold(n_splits=3, shuffle=True)

    for fold, (train_idx, test_idx) in enumerate(kf.split(train_df, Y)):
        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
        X_train, X_valid = train_df[train_idx], train_df[test_idx]

        y_train, y_valid = Y[train_idx], Y[test_idx]

        model = build_model()

        # A callback to save the model
        callback0 = tf.keras.callbacks.ModelCheckpoint(f"M1_PressurePreModel{fold+1}.h5", 
                                               monitor='val_m1_out_loss',save_best_only=True, verbose=1)
        callback1 = tf.keras.callbacks.ModelCheckpoint(f"C_PressurePreModel{fold+1}.h5", 
                               monitor='val_C_out_loss',save_best_only=True, verbose=1)

        callbacks = [lr_callback, callback0, callback1]

        his = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE, callbacks=callbacks) 

In [ ]:
models_paths = ['../input/ventilator-pressure-predictor-tpu/M1_PressurePreModel1.h5',
                '../input/ventilator-pressure-predictor-tpu/M1_PressurePreModel2.h5']
 
models = [tf.keras.models.load_model(model_path) for model_path in models_paths]

In [ ]:
test_data = pd.read_csv(test_path)
test_data = preProcess(test_data)

test_data = dropCols(test_data, cols_2_drop)
test_data = rc.transform(test_data)
test_data = test_data.reshape(-1, 80, test_data.shape[-1])

In [ ]:
p = []

for model in models:
    p.append(model.predict(test_data, verbose=1))

In [ ]:
predictions_table0 = np.concatenate([p[0][0], p[1][0]], axis=-1)
predictions_table1 = np.concatenate([p[0][1], p[1][1]], axis=-1)

In [ ]:
median_pre0 = np.median(predictions_table0, axis = -1)
median_pre1 = np.median(predictions_table1, axis = -1)

In [ ]:
unique_pressures = np.unique(Y)

In [ ]:
sorted_pressures = np.sort(unique_pressures)

PRESSURE_STEP = (unique_pressures[1] - unique_pressures[0]).item()

PRESSURE_MIN = sorted_pressures[0].item()
PRESSURE_MAX = sorted_pressures[-1].item()

In [ ]:
PRESSURE_STEP,PRESSURE_MIN,PRESSURE_MAX

In [ ]:
rounding_pre0 = np.round((median_pre0 - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN
rounding_pre1 = np.round((median_pre1 - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN
#Rounding With Median of Model

In [ ]:
rounding_pre0.shape,rounding_pre1.shape

In [ ]:
clipped_pre0 = np.clip(rounding_pre0, PRESSURE_MIN, PRESSURE_MAX).reshape(-1,1)
clipped_pre1 = np.clip(rounding_pre1, PRESSURE_MIN, PRESSURE_MAX).reshape(-1,1)

In [ ]:
clipped_pre0.shape,clipped_pre1.shape

In [ ]:
submission_file = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")

submission_file['pressure'] = clipped_pre0
submission_file.to_csv('submission0.csv', index=False)

submission_file['pressure'] = clipped_pre1
submission_file.to_csv('submission1.csv', index=False)

In [ ]:
submission_file